# First part: Weave insert first list into second

### usage
```
weave [11,12] [0,1,2,3]
```

### output
```
[0,11,1,12,2,11,3]
```

**What we want to do here** is interleave the first list into the second, until the _second_ one runs out. Well, the runtime `zip` function says it will, given 2 lists as input, return a list of the corresponding pair, with the first element of the pair being from the first list. Well, we want it the other way around and, even though we could simply invert the order when feeding them to the function, we'll use `zipWith` with our own function just to keep the input intact. So let's try the following:

## Function: `weave`

In [86]:
la = [11,12]
lb = [0,1,2,3]
zipWith (\x y -> [y, x]) (cycle la) lb

[[0,11],[1,12],[2,11],[3,12]]

Just to clarify, we `cycle` the first list to make sure we stop just after the end of the second one. So, that gave us something good to start with. First of all, we don't want the pairs organized into sublists of their own, so we can `concat` that:

In [87]:
concat $ zipWith (\x y -> [y, x]) (cycle la) lb

[0,11,1,12,2,11,3,12]

Cool, we're almost there. We just need to get rid of the trailing element which shouldn't be there; remember that the head and last elements of the second list must enclose the resulting list. So we can just pick the `init` of the result:

In [88]:
init $ concat $ zipWith (\x y -> [y, x]) (cycle la) lb

[0,11,1,12,2,11,3]

Got'em. So our only function effectively becomes:

In [89]:
weave a b = init $ concat $ zipWith (\x y -> [y, x]) (cycle a) b

Looks elegant. We use it like so:

In [90]:
weave la lb

[0,11,1,12,2,11,3]

Different inputs:

In [91]:
weave "abcd" "[]" -- first longer than the second

"[a]"

In [92]:
weave ["*"] ["(2+3)", "(4-5)", "(3+7)"] -- second longer than the first

["(2+3)","*","(4-5)","*","(3+7)"]

In [93]:
concat $ weave ["*"] ["(2+3)", "(4-5)", "(3+7)"] -- same thing, more readable

"(2+3)*(4-5)*(3+7)"

# Second part: Bracket insert first list into second

### usage
```
bracket "abc" "()"
```

### output
```
["(a)", "(b)", "(c)"]
```

**We can already see** that a simple `zip` won't do. But we _do_ know that a tuple is one thing and a list of tuples is still a list of those one things... If you didn't get the hint, just bear with me for a moment. Let's first try to generate a list of tuples containing pairs of element from a list:

In [94]:
let { zipPairs [xa, xb]   = [(xa, xb)];
        zipPairs (xa:xb:xs) = (xa, xb):zipPairs xs }

Remember that `[xa, xb]` is just syntax candy for `xa:xb:[]`. Let's try it out:

In [95]:
zipPairs "()[]"

[('(',')'),('[',']')]

Awesome.

**Yes**, it'll have to be an even number of elements, but that's a natural implication. We'll assume the input will respect these guidelines, there's no need for robustness here. Notice that now, having a list of tuples, we can, again, capitalize on the functionality of `zipWith`, with a lil' modification to our lambda function:

In [96]:
lx = "abc"
ly = "()"
zipWith (\x (ya, yb) -> [ya, x, yb]) lx (zipPairs ly)

["(a)"]

Hmm, close. It looks like we're a few elements short. The reason why we didn't simply `cycle` the first list is that the function must run until _both_ lists exhaust. That means that if the first is bigger than the half of the second, we cycle the first until we end the second and vice-versa. This is somewhat simple, but we need to know which list is bigger to do the right thing. A little `if&else` will do the trick:

## Function: `bracket`

In [97]:
bracket a b = if length a >= (length b `div` 2) then
                            zipWith (\x (ya, yb) -> [ya, x, yb]) a (cycle (zipPairs b))
                        else
                            zipWith (\x (ya, yb) -> [ya, x, yb]) (cycle a) (zipPairs b)

Let's give it a go:

In [98]:
bracket lx ly

["(a)","(b)","(c)"]

Beautiful. Let's try a few variations of inputs:

In [99]:
bracket "ab" "()" -- both lists the same size

["(a)","(b)"]

In [100]:
bracket "+-" "123456" -- second longer than first

["1+2","3-4","5+6"]

In [101]:
bracket ["2+3", "4-5", "6+7"] ["(", ")"] -- fisrt longer than second

[["(","2+3",")"],["(","4-5",")"],["(","6+7",")"]]

In [102]:
concatMap concat $ bracket ["2+3", "4-5", "6+7"] ["(", ")"] -- same thing, more readable

"(2+3)(4-5)(6+7)"

## e l e g a n c e

This oughta' do the trick! Plus, the complete program, with a few checks for blank inputs (which we didn't include here)  is about 8~10 lines long. If you found this hard to follow, you should check out [lambdas](http://learnyouahaskell.com/higher-order-functions#lambdas) and [function application](http://learnyouahaskell.com/higher-order-functions#function-application). The [list functions library](https://hackage.haskell.org/package/base-4.9.0.0/docs/Data-List.html) documents all the functions used here, if you felt unsure at some point.